# Prediction of Singapore's housing price using simple regression based models
## Regression Modelling notebook

This project is about the prediction of resale price of a house located in singapore, using simple regression model.   
This notebook is only for the purpose of building Regression Model.  
Analytical solution to least squares regression is given by:  
<img src="../img/least_square_sol.png"></img>  
Which I will not go into details here.  

This note book will show the process of building regression model(s) to predict the housing price in Singapore.  
Which is in the area of Machine Learning.  

# Problem Statement

The goal of this project is to build a regression model, using data contained in the [datasets](../datasets) folder. The model should be able to make an accurate prediction of the resale price (`resale_price`) of the house, for every house id (`Id`) that appeared in the [test set](../datasets/test.csv).  
Success will be evaluated based on common evaluation metrics such as Mean Absolute Error (MAE) and Mean Square Error (MSE), apart from scores.

Motivation:  
While this is a toy project for the purpose of learning, it shows the importance of prediction models.  
House owners who are looking to sale their property, property agents, those seeking to purchase a house, all stand to benefit from this model.  

Contents:  
1. [Single Model](#Single-Model)   
    1.1 [Preprocessing](#Preprocessing)  
    1.2 [Linear Regression](#Linear-Regression)  
    1.3 [Lasso Regression](#Lasso-Regression)  
    1.4 [Ridge Regression](#Ridge-Regression)  
    1.5 [Elastic Net](#Elastic-Net-Regression)   
    1.6 [Model Evaluation](#Model-Evaluation)  
2. [Combined Models]  
    2.1 [Data Selection]  
    2.2 [Preprocessing]   
    2.3 [Regression Model]  
    2.4 [Grid Search Cross Validation]  
    2.5 [Model Evaluation]  

In [1]:
import functools
import numpy as np
import pandas as pd
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, ElasticNet, LassoCV, Lasso, Ridge, RidgeCV
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn import metrics

## Single Model

This section is the most basic model created.   
I will start with simple preprocessing, then build simple models as well as perform Grid Search Cross Validation.  
Then, these models with tunned paprameters will be examined and evaluated.

In [2]:
# read in the cleaned data used in model creation
df = pd.read_csv('../datasets/reduced_train.csv')
df.head()

,id,flat_type,floor_area_sqm,flat_model,resale_price,Tranc_Year,mid,hdb_age,max_floor_lvl,planning_area,Mall_Nearest_Distance,Hawker_Nearest_Distance,mrt_nearest_distance,bus_interchange,mrt_interchange,cutoff_point
0,88471,4 ROOM,90.0,Model A,680000.0,2016,11,15,25,Kallang,1094.090418,154.753357,330.083069,0,0,224
1,122598,5 ROOM,130.0,Improved,665000.0,2012,8,34,9,Bishan,866.941448,640.151925,903.659703,1,1,232
2,170897,EXECUTIVE,144.0,Apartment,838000.0,2013,14,24,16,Bukit Batok,1459.579948,1762.082341,1334.251197,1,0,188
3,86070,4 ROOM,103.0,Model A,550000.0,2012,3,29,11,Bishan,950.175199,726.215262,907.453484,1,1,253
4,153632,4 ROOM,83.0,Simplified,298000.0,2017,2,34,4,Yishun,729.771895,1540.151439,412.343032,0,0,208


In [3]:
df.shape

(149772, 16)

In [4]:
df.isnull().sum()

id                         0
flat_type                  0
floor_area_sqm             0
flat_model                 0
resale_price               0
Tranc_Year                 0
mid                        0
hdb_age                    0
max_floor_lvl              0
planning_area              0
Mall_Nearest_Distance      0
Hawker_Nearest_Distance    0
mrt_nearest_distance       0
bus_interchange            0
mrt_interchange            0
cutoff_point               0
dtype: int64

## Preprocessing

Split the dataset as categorical and numerical sets.  
This is a very basic and simple preprocessing, where the categorical features will go through One Hot Encoding, numerical features will go through Standard Scaler.

In [5]:
categorical_col = ['flat_type', 'flat_model', 'Tranc_Year', 'planning_area', 'bus_interchange', 'mrt_interchange']
numerical_col = ['floor_area_sqm', 'mid', 'hdb_age', 'max_floor_lvl', 'Mall_Nearest_Distance', 'Hawker_Nearest_Distance', 'mrt_nearest_distance', 'cutoff_point']

In [6]:
# This is the target of the prediction
y = df.resale_price

In [7]:
# These are the predictors used to predict the resale price
X = df.drop(columns=['resale_price'])

In [8]:
# make a pipeline to apply standard scaler on selected features
numeric_transformer = Pipeline(
    steps=[("scaler", StandardScaler())]
)

In [9]:
# make a pipeline to apply one hot encoding on selected features
categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore", drop='first')),
    ]
)

In [10]:
# the above 2 steps forms the colmun transformer, which is used for the preprocessing step.

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numerical_col),
        ("cat", categorical_transformer, categorical_col),
    ]
)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

## Linear Regression

This section covers the building of a simple linear regression model.  

In [12]:
# Build a pipeline for regression model
lin_reg = Pipeline(
    steps=[("preprocessor", preprocessor), ("regressor", LinearRegression())]
)

In [13]:
# Fit the model with training data
lin_reg.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['floor_area_sqm', 'mid',
                                                   'hdb_age', 'max_floor_lvl',
                                                   'Mall_Nearest_Distance',
                                                   'Hawker_Nearest_Distance',
                                                   'mrt_nearest_distance',
                                                   'cutoff_point']),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['flat_type', 'flat_model',
                                                   'Tranc_Year',
                                                   'planning_area',
                                                   'bus_interchange',
                                                   'mrt_interchange'])])),
                ('regressor', LinearRegression())])

In [14]:
print(f"The train score is: {round(lin_reg.score(X_train, y_train), 6)}")
print(f"The test score is: {round(lin_reg.score(X_test, y_test), 6)}")

The train score is: 0.891913
The test score is: 0.891237


This is a reseasonable model with very close train and test scores.   
This means that this is a decent model!  
Now, I will further investigate this model using cross validation.

In [15]:
# Build a regression model with cross validation
folds = KFold(n_splits = 5, shuffle = True, random_state = 0)
scores = cross_val_score(lin_reg, X_train, y_train, scoring='r2', cv=folds)

In [16]:
scores

array([0.8913561 , 0.89231941, 0.89244822, 0.8928205 , 0.88969684])

Seems that the cross validation score is pretty good and very consistent.  
This is indicated by the close match between the different folds.  
Below is a simpler way to do it.  
```console
scores = cross_val_score(lin_reg, X_train, y_train, scoring='r2', cv=5)
scores      
```
Okay, this is acceptable.  
Now, time to try the other methods!

## Lasso Regression

Lasso (least absolute shrinkage and selection operator; also Lasso or LASSO), is a regression analysis method.  
It is also termed as L1 regularization.  
It is able to shrink some coefficients to 0.  
Lasso Regression is given by:  
<img src="../img/lasso.png"></img>  
Using my own naive understanding, lasso is solving for optimal point.  
The solution to these optimal points appear when elliptical contours touch the constraints.   
Solutions exists at vertices, just like Linear Programming.  
These vertices just so happens to lie on axis.  
Which then results in at least 1 coefficent to be shrinked to 0.  
Amazing mathematics at work!

In [17]:
# Build a pipeline for regression model
# default setting did not converge
lasso = Pipeline(
    steps=[("preprocessor", preprocessor), ("Lasso", Lasso(max_iter=10000))]
)

In [18]:
lasso.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['floor_area_sqm', 'mid',
                                                   'hdb_age', 'max_floor_lvl',
                                                   'Mall_Nearest_Distance',
                                                   'Hawker_Nearest_Distance',
                                                   'mrt_nearest_distance',
                                                   'cutoff_point']),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['flat_type', 'flat_model',
                                                   'Tranc_Year',
                                                   'planning_area',
                                                   'bus_interchange',
                                                   'mrt_interchange'])])),
                ('Lasso', Lasso(max_iter=10000))])

In [19]:
print(f"The train score is: {round(lasso.score(X_train, y_train), 6)}")
print(f"The test score is: {round(lasso.score(X_test, y_test), 6)}")

The train score is: 0.891912
The test score is: 0.891241


Hmmmm, this is very close match!  
Take a look at cross validation?  

In [20]:
lasso_CV = Pipeline(
    steps=[("preprocessor", preprocessor), ("Lasso CV", LassoCV(max_iter=10000, cv=5))]
)

In [21]:
lasso_CV.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['floor_area_sqm', 'mid',
                                                   'hdb_age', 'max_floor_lvl',
                                                   'Mall_Nearest_Distance',
                                                   'Hawker_Nearest_Distance',
                                                   'mrt_nearest_distance',
                                                   'cutoff_point']),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['flat_type', 'flat_model',
                                                   'Tranc_Year',
                                                   'planning_area',
                                                   'bus_interchange',
                                                   'mrt_interchange'])])),
                ('Lasso CV', LassoCV(cv=5, max_iter=10000))])

In [22]:
print(f"The train score is: {round(lasso_CV.score(X_train, y_train), 6)}")
print(f"The test score is: {round(lasso_CV.score(X_test, y_test), 6)}")

The train score is: 0.886419
The test score is: 0.886501


This is in fact lower than simply fitting using `.Lasso()`.  
Hmmm....   
This maybe becasue the model is in fact trained with smaller training data size?  
Anyways, continue to play with different alpha values.

In [24]:
# Specify range of hyperparameters to tune
lasso_params = {'alpha':[1, 10, 100, 1000]}

In [25]:
# Perform grid search using GridSearchCV()
grid_search = GridSearchCV(Lasso(max_iter=10000),
                           param_grid=lasso_params,
                           cv=5,
                           verbose = 3,
                           return_train_score=True
                           )

lasso_grid_search_CV = Pipeline(
    steps=[("preprocessor", preprocessor), ("Lasso", grid_search)]
) 

Warning, you may not want to run the line below...

In [26]:
lasso_grid_search_CV.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5] END .......alpha=1;, score=(train=0.892, test=0.893) total time= 1.8min
[CV 2/5] END .......alpha=1;, score=(train=0.892, test=0.891) total time= 2.2min


/home/zhiming/miniconda3/envs/dsi-sg/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:609: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 11515741251960.375, tolerance: 196720491074.18277
  model = cd_fast.sparse_enet_coordinate_descent(


[CV 3/5] END .......alpha=1;, score=(train=0.892, test=0.892) total time= 2.6min
[CV 4/5] END .......alpha=1;, score=(train=0.892, test=0.892) total time= 2.3min
[CV 5/5] END .......alpha=1;, score=(train=0.892, test=0.890) total time= 1.5min
[CV 1/5] END ......alpha=10;, score=(train=0.891, test=0.893) total time=  22.6s
[CV 2/5] END ......alpha=10;, score=(train=0.892, test=0.891) total time=  45.3s
[CV 3/5] END ......alpha=10;, score=(train=0.892, test=0.891) total time=   9.9s
[CV 4/5] END ......alpha=10;, score=(train=0.892, test=0.892) total time=  38.7s
[CV 5/5] END ......alpha=10;, score=(train=0.892, test=0.890) total time=  10.1s
[CV 1/5] END .....alpha=100;, score=(train=0.886, test=0.888) total time=   0.7s
[CV 2/5] END .....alpha=100;, score=(train=0.886, test=0.886) total time=   1.4s
[CV 3/5] END .....alpha=100;, score=(train=0.886, test=0.885) total time=   0.7s
[CV 4/5] END .....alpha=100;, score=(train=0.886, test=0.885) total time=   1.3s
[CV 5/5] END .....alpha=100;

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['floor_area_sqm', 'mid',
                                                   'hdb_age', 'max_floor_lvl',
                                                   'Mall_Nearest_Distance',
                                                   'Hawker_Nearest_Distance',
                                                   'mrt_nearest_distance',
                                                   'cutoff_point']),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['flat_type', 'flat_model',
                                                   'Tranc_Year',
                                                   'planning_area',
                                                   'bus_interchange',
                                                   'mrt_interchange'])])),
                ('Lasso',
                 GridSearchCV(cv=5, estimator=Lasso(max_iter=10000),
                              param_grid={'alpha': [1, 10, 100, 1000]},
                              return_train_score=True, verbose=3))])

How naive...   
I started with 'lasso_params = {'alpha':[0.01, 0.1, 1, 10, 100]}'
All alpha below 1 runs into convergence issues...  
For `alpha = 0.01`, the scores from grid search are around `(train=0.892, test=0.893+/-0.003)`.  
They also took a total of 2.6 minutes to run 10000 iterations and then conclude that the solution did not converge.  
This is not acceptable, as it takes too long for a simple model.  
So the final alphas to search are in the range of 1 to 100.
Alpha is the constant that multiplies the L1 term, controlling regularization strength.  
So the higher the Alpha, the more aggressive the penalization for complex model.  
The higher the penalization, the faster the convergence, as the area bounded by the constraints are larger.  
This is evident from the difference in total time used to compute the models.

In [29]:
lasso_grid_search_CV[1].best_score_

0.8917186127412082

In [31]:
# cv results
cv_results = pd.DataFrame(lasso_grid_search_CV[1].cv_results_)
cv_results[['mean_fit_time', 'param_alpha', 'mean_test_score', 'mean_train_score']]

,mean_fit_time,param_alpha,mean_test_score,mean_train_score
0,124.816828,1,0.891719,0.891932
1,25.302761,10,0.891458,0.891685
2,1.000720,100,0.885856,0.886109
3,0.172793,1000,0.817216,0.817400


By looking the the results of the cross validation results, alpha = 1 gives the best result. 
The result becomes worse when alpha increases.  
There could be a possibility that a minima exists between 0 and 1.  
But I think the difference is minor and it doesn't warrant the time spent on searching for it.  
There is also a possibility (very likely in fact) that if alpha = 0 is used, it will give the best result.  
Which is simply using linear regression, like the previous section.  
If Lasso Regression must be used, I will want to use Alpha >= 100.  
This is becasue the model is solved much faster, with very small loss in performace.  

## Ridge Regression

Ridge regression is a model tuning method that is used to analyse any data that suffers from multicollinearity.   
This method performs L2 regularization.  
It is UNABLE to shrink coefficients to 0.  
Ridge Regression is given by:  
<img src="../img/ridge.png"></img>  
Using my own naive understanding, Ridge is also solving for optimal point.  
The solution to these optimal points appear when elliptical contours touch the constraints, like Lasso.   
Unlike lasso, the contraint region from Ridge are cirular.   
The coefficients will be unable to shrink to 0.  
Amazing mathematics at work again!

In [29]:
# Build a pipeline for regression model
# default setting did not converge
ridge = Pipeline(
    steps=[("preprocessor", preprocessor), ("Ridge", Ridge())]
)

In [30]:
ridge.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['floor_area_sqm', 'mid',
                                                   'hdb_age', 'max_floor_lvl',
                                                   'Mall_Nearest_Distance',
                                                   'Hawker_Nearest_Distance',
                                                   'mrt_nearest_distance',
                                                   'cutoff_point']),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['flat_type', 'flat_model',
                                                   'Tranc_Year',
                                                   'planning_area',
                                                   'bus_interchange',
                                                   'mrt_interchange'])])),
                ('Ridge', Ridge())])

In [31]:
print(f"The train score is: {round(ridge.score(X_train, y_train), 6)}")
print(f"The test score is: {round(ridge.score(X_test, y_test), 6)}")

The train score is: 0.891556
The test score is: 0.890854


It have a slightly worse performance compared to both Lasso and Linear Regression.  
Now investigate with CV.

In [37]:
ridge_CV = Pipeline(
    steps=[("preprocessor", preprocessor), ("Ridge CV", RidgeCV(alphas=[0.1, 1, 10, 100, 1000], store_cv_values=True, alpha_per_target=True))]
)

In [38]:
ridge_CV.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['floor_area_sqm', 'mid',
                                                   'hdb_age', 'max_floor_lvl',
                                                   'Mall_Nearest_Distance',
                                                   'Hawker_Nearest_Distance',
                                                   'mrt_nearest_distance',
                                                   'cutoff_point']),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['flat_type', 'flat_model',
                                                   'Tranc_Year',
                                                   'planning_area',
                                                   'bus_interchange',
                                                   'mrt_interchange'])])),
                ('Ridge CV',
                 RidgeCV(alpha_per_target=True, alphas=[0.1, 1, 10, 100, 1000],
                         store_cv_values=True))])

In [39]:
print(f"The train score is: {round(ridge_CV.score(X_train, y_train), 6)}")
print(f"The test score is: {round(ridge_CV.score(X_test, y_test), 6)}")

The train score is: 0.891919
The test score is: 0.891239


In [48]:
cv_results= pd.DataFrame(ridge_CV[1].cv_values_, columns=['alpha=0.1', 'alpha=1', 'alpha=10', 'alpha=100', 'alpha=1000'])
cv_results.shape

(119817, 5)

In [47]:
cv_results.head()

,alpha=0.1,alpha=1,alpha=10,alpha=100,alpha=1000
0,2.748284e+09,2.755258e+09,2.813214e+09,3.193140e+09,4.817755e+09
1,1.418368e+09,1.423038e+09,1.457278e+09,1.611593e+09,1.377079e+09
2,1.991506e+07,1.945613e+07,1.547463e+07,8.832042e+05,1.783817e+03
3,4.380845e+09,4.388701e+09,4.462062e+09,5.059141e+09,8.126840e+09
4,1.690679e+09,1.691574e+09,1.703623e+09,1.891572e+09,3.635783e+09


In [52]:
ridge_CV[1].alpha_

0.1

In [50]:
ridge_CV[1].best_score_

-2222773760.7176504

In [51]:
ridge_CV[1].n_features_in_

75

The best Alpha selected is 0.1.  
This means the default Ridge() is good enough.  
I will now move on to Elastic Net.

## Elastic Net Regression

Elastic Net Regression is a combination of Lasso (L1) and Ridge (L2).  
Maybe that's why it is called elastic net?

## Model Evaluation

Here, I will take a (slightly) closer look at the four models used.  
I will look at the scores, where the default is the r-squared value, first.  
I will peek at the coefficients.  
Then I will make inference using the models, and take a look at the MSE, RMSE, AME.

Lasso is using 'diamond' shaped constriants.  
Ridge is using 'circle' shaped constraints.  
Once the elliptical contours hits the constraints, optimality is reached.  
But becasue 'diamond' have vertices on axis, it will shrink at least 1 coefficient to 0.

<img src="../img/lasso_vs_ridge.png" width="500" height="600"></img>  

Ridge, on the other hand will not, as optimal point will not lie on axis, which are representing features.  
(Unless the whole B_hat lies on axis, resulting in a trivial solution? This is just my guess)

In [86]:
## failed attempt to do GridSearchCV on linear regression
### problem identified to be preprocessor can't contain less features than specified
## keeping for future investigation

# build a function for preprocessor of dataset
# def get_present_col_subset(selected_columns, df):
#     '''return the currently available colmuns
#     args:
#         selected_colmuns: list of colmuns to select from
#         df: dataframe being used currently
#     return:
#         subset_col: colmuns present in selected colmuns
#     Note: Code taken from user: srggrs at https://github.com/scikit-learn/scikit-learn/issues/19014
#     '''
#     # get the intersecton of present and known-infrequent columns
#     present_columns = df.columns
#     return [col for col in present_columns if col in selected_columns]

In [85]:
## failed attempt to do GridSearchCV on linear regression
### problem identified to be preprocessor can't contain less features than specified

# build the preprocessor for the dataset
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numeric_transformer, functools.partial(get_present_col_subset, numerical_col)),
#         ('cat', categorical_transformer, functools.partial(get_present_col_subset, categorical_col))
#     ]
# )

In [83]:
## failed attempt to do GridSearchCV on linear regression
### problem identified to be preprocessor can't contain less features than specified

# # Prepare the training data to be 5 folds
# folds = KFold(n_splits = 5, shuffle = True, random_state = 0)

# # Specify range of hyperparameters to tune
# hyper_params = [{'n_features_to_select': list(range(1, len(X_train.columns)))}]

# # Build Feature ranking with recursive feature elimination
# lin_reg.fit(X_train,y_train)
# rfe = RFE(lin_reg)  

# # Perform grid search using GridSearchCV()
# lin_reg_cv = GridSearchCV(estimator = lin_reg, 
#                         param_grid = hyper_params, 
#                         scoring= 'r2', 
#                         cv = folds, 
#                         verbose = 1,
#                         return_train_score=True)      

# # Fit the model
# lin_reg_cv.fit(X_train, y_train) 